# 1st

## Model Explore

In [ ]:
!pip install -q transformers accelerate bitsandbytes huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
notebook_login()

## more model

In [ ]:
torch.cuda.is_available()

True

In [ ]:
model_id = "meta-llama/Llama-2-7b-hf"

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Load model with 8-bit precision (needs less memory)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_8bit=True,  # For lower RAM usage (needs `bitsandbytes`)
    torch_dtype=torch.float16,
)


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
def answer(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    # output = model.generate(**inputs, max_new_tokens=200)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        eos_token_id=tokenizer.eos_token_id)
    # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
prompt = " I have knee pain which medicine should i take?"
answer(prompt)

" I have knee pain which medicine should i take?\nWhat type of pain is it?\nHow long has it been going on?\nWhat makes the pain worse?\nDo you have any other symptoms?\nDo you have any history of injury or illness?\nWhat do you do about it?\nDo you take any medications for it?\nWhat other medications do you take?\nWhat is your job?\nWhat do you do for fun?\nWhat is your favorite food?\nDo you have any other questions?\nIf so, ask them.\nI'll explain more about these questions later in the book.\n11\nThe first thing you need to do is to stop worrying about the pain. It is not as bad as you think"

In [ ]:
prompt = "I have headache which medicine should i take?"
answer(prompt)

"I have headache which medicine should i take?\nAnswers from experts on medicine for headache. First: If you've had a headache for 2 weeks, it's not a headache, it's a brain tumor. I would suggest you get to an ER right away. Second: It depends on the type of headache you have. The best medicine is to find the cause. Third: I have a headache that lasts 10 days. I took aspirin and it did not help. I took ibuprofen and it did not help. What else can I take?\nWhat is the best medicine for a headache?\nI have a headache. What is the best medicine for it?\nCan I"

# detection

In [ ]:
from transformers import  pipeline
import torch
import nltk

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import json
import pandas as pd
import random

# ---------------------------
# Step 3: Load benchmarking data and select 50 random questions
# ---------------------------
with open("benchmarking.json", "r") as f:
    data = json.load(f)

# Select up to 50 random entries for quicker benchmarking
sampled = random.sample(data, min(len(data), 50))

# ---------------------------
# Step 4: Generate model answers and save to CSV
# ---------------------------
records = []
for entry in sampled:
    q = entry["question"]
    std = entry["answer"]
    prompt_text = f"Question: {q}\nAnswer:"  # instruction to the model
    gen = answer(prompt_text)
    records.append({
        "question": q,
        "model_answer": gen.strip(),
        "standard_answer": std
    })

In [ ]:
# Save results for later detection
df = pd.DataFrame(records)
df.to_csv("model_vs_standard.csv", index=False)
print("Saved model_vs_standard.csv")


Saved model_vs_standard.csv


In [ ]:

# Setup NLI verification pipeline

nli = pipeline(
    "text-classification",
    model="ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
)

# nltk.download('punkt')  # for sentence tokenization





config.json:   0%|          | 0.00/703 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# ---------------------------
# Step 6: Define detection function using standard answers as context
# ---------------------------
def detect_hallucinations(answer: str, context: str) -> list:
    """
    Splits the generated answer into sentences and verifies each
    against the standard answer (context) using NLI.
    Returns a list of {sentence, label, score}
    """
    sentences = nltk.sent_tokenize(answer)
    results = []
    for sent in sentences:
        nli_input = context + " [SEP] " + sent
        res = nli(nli_input)[0]
        results.append({
            "sentence": sent,
            "label": res["label"],
            "score": res["score"]
        })
    return results

In [ ]:
records[0]

{'question': 'What should you do if you suspect a concussion?',
 'model_answer': 'Question: What should you do if you suspect a concussion?\nAnswer: Call your doctor or go to the emergency room.\nWhat should you do if you suspect a concussion?\nCall your doctor or go to the emergency room.\nThis is a new question on the SAT. It’s a good question to review in case you are preparing for the test.\nWhat should you do if you suspect a concussion? Call your doctor or go to the emergency room.\nPrevious Post What is the best way to prepare for the SAT?\nNext Post Which of the following is not true about the SAT?',
 'standard_answer': 'Seek medical attention immediately, rest, and avoid activities that could worsen your condition, such as physical exertion and using digital screens.'}

In [ ]:
# ---------------------------
# Step 7: Run detection on saved model outputs
# ---------------------------
all_detections = []
for rec in records:
    dets = detect_hallucinations(rec["model_answer"], rec["standard_answer"])
    all_detections.append({
        "question": rec["question"],
        "detections": dets
    })


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
for item in all_detections:
    print(f"Question: {item['question']}")
    for det in item['detections']:
        print(f"  • Label: {det['label']}, Confidence: {det['score']:.2f}")
    print()


Question: What should you do if you suspect a concussion?
  • Label: neutral, Confidence: 1.00
  • Label: neutral, Confidence: 0.84
  • Label: neutral, Confidence: 0.99
  • Label: neutral, Confidence: 0.97
  • Label: neutral, Confidence: 1.00
  • Label: neutral, Confidence: 1.00
  • Label: neutral, Confidence: 0.99
  • Label: neutral, Confidence: 0.97
  • Label: neutral, Confidence: 0.99
  • Label: neutral, Confidence: 0.95

Question: What are signs that you need to visit an optometrist?
  • Label: neutral, Confidence: 0.98
  • Label: neutral, Confidence: 0.99
  • Label: neutral, Confidence: 0.98
  • Label: neutral, Confidence: 0.99
  • Label: neutral, Confidence: 0.99
  • Label: neutral, Confidence: 0.96

Question: What should I do if I suspect food poisoning?
  • Label: neutral, Confidence: 0.99
  • Label: neutral, Confidence: 0.86
  • Label: neutral, Confidence: 0.96
  • Label: neutral, Confidence: 0.98
  • Label: neutral, Confidence: 1.00
  • Label: neutral, Confidence: 1.00
  • La

In [ ]:
# Step 9: Aggregate and print overall results
# ---------------------------
for item in all_detections:
    labels = [d["label"] for d in item["detections"]]
    if "CONTRADICTION" in labels:
        overall = "Hallucination detected"
    elif "NEUTRAL" in labels:
        overall = "Potentially uncertain (no direct contradiction)"
    else:
        overall = "No hallucination detected"
    print(f"Question: {item['question']}\n  → {overall}\n")

Question: What should you do if you suspect a concussion?
  → No hallucination detected

Question: What are signs that you need to visit an optometrist?
  → No hallucination detected

Question: What should I do if I suspect food poisoning?
  → No hallucination detected

Question: What are the uses of calcium channel blockers?
  → No hallucination detected

Question: What should I do if I have swollen gums?
  → No hallucination detected

Question: What should I take for gas and bloating?
  → No hallucination detected

Question: What is the use of anticoagulant medications?
  → No hallucination detected

Question: What should you do if you experience side effects from a medication in Australia?
  → No hallucination detected

Question: How can you improve bone density?
  → No hallucination detected

Question: How does Australia manage the regulation of prescription opioids?
  → No hallucination detected

Question: What should I take if I am feeling nauseous?
  → No hallucination detected


1. **Input Preparation**
   1.1. I create a list of examples, each a dict with:
     • `text` (premise)
     • `text_pair` (hypothesis)
   1.2. I call the pipeline on that list:

   ```python
   results = nli(pairs)
   ```

2. **Tokenization & Batch Collation**
   2.1. **Concatenate** each pair into one sequence:

   ```
   <s> PREMISE </s> </s> HYPOTHESIS </s>
   ```

  

3. **Joint Encoding via RoBERTa**
   3.1. **Embed** input IDs into vectors
   3.2. **Self-Attention Layers**
     • Every token attends to every other token in the concatenated sequence
     • This enables direct comparison


4. **Classification Head**
   4.1.  small feed-forward network
   4.2. It produces **3 logits**:

     • ENTAILMENT

     • NEUTRAL


     • CONTRADICTION

5. **Softmax & Label Selection**
   5.1. softmax to the logits → probabilities that sum to 1
   5.2. pick the label with the highest probability

6. **Return Results**
   6.1. output a list of dicts in the same order as my input, each with:
     • `label` (ENTAILMENT/NEUTRAL/CONTRADICTION)
     • `score` (the probability)





**model predictions, each given as Label Confidence (e.g. neutral 62%, entailment 52%, …, entailment 96%), showing for each input pair the predicted relation and how certain the model is.**

In [ ]:
# Save detections for analysis
with open("hallucination_detections.json", "w") as f:
    json.dump(all_detections, f, indent=2)
print("Saved hallucination_detections.json")